# EXTRACT 
## Fetching Data

In [ ]:
""" Load in modules we will need """
import os
import gc
import pymssql
import numpy as np 
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime as dt
load_dotenv()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#### "Reading" from Bronze Layer 

In [ ]:
"""
Retrieve all the csv file names from a given path

Parameters: 
path (string)- Path we are looking through

Returns: a list of .csv files
"""
def grab_files(path: str) -> list:
    return [file for file in os.listdir(path) if file.endswith(".csv")]

"""
Creates DataFrames for all data of a specific entity in "Bronze layer"

Parameters: 
path (string) - The "container" for the data 

Returns: a list of .csv files
"""
def gen_dfs(ospath = 'BRONZE_LAYER', path:str =''):    
    dfs = []
    try:
        data_dir = f'{os.environ[ospath]}{path + "/" if path != "" else path}'
        files = grab_files(data_dir)
        for f in files:
            df = pd.read_csv(data_dir+f)
            df.name = f.split('__')[1]
            dfs.append(df)
    except FileNotFoundError:
        print("Error with pathing")
    except:
        print('Error Occurred while extracting data from the file')
    return dfs        #create the dataframe


# TRANSFORM
## Cleaning Functions

In [ ]:
"""
Will Attempt to Drop all declared columns from a DataFrame

Parameters: 
df - a singular Data Frame
cols - a list of strings containing column attributes to be dropped

Returns: Nothing
"""
def drop_dfcols(df, cols:list):
    try:
        df.drop(columns = cols, inplace = True)
    except KeyError:
        print(f'fail to remove{cols}')

"""
Will Attempt to Drop all declared columns from a List of DataFrame

Parameters: 
df - a List of Data Frame
cols - a list of strings containing column attributes to be dropped 

Returns: Nothing
"""
def drop_dfs_col(dfs, cols: list):
    try:
        for df in dfs:
            drop_dfcols(df,cols) 
    except KeyError:
        print('no columns left')
    


In [ ]:
"""
Remove duplicate records from the DataFrame
"""
def rm_dupe(df):
    df.drop_duplicates(inplace= True)

"""
Drops the columns that have null vals
or we fill them will default values if we want
"""
def rm_nulls(df, col, default_val = None):
    if default_val:
        df[col].fillna(default_val, inplace = True)
    else:
        df.dropna(subset =[col], inplace= True) 

"""
Checks to see if any column of a DataFrame contains Nulls
-best to do this after dropping useless columns [faster]
"""
def cols_with_null(df):
    has_null = []
    for col in df.columns:
        print(f'{col} : {df[col].isnull().values.any()}')
        if df[col].isnull().values.any():
            has_null.append(col)
    return has_null

"""
Parameters:
dfs: List of DataFrames 
Return: Merged DataFrames 
"""
def merge_dfs(dfs: list):
    name_attr = dfs[0].name if hasattr(dfs[0], "name") else None
    dfs = pd.concat(dfs, ignore_index=True)        #put all the data into one table
    if name_attr:
        dfs.name = name_attr
    return dfs

In [ ]:
"""
Checks to see if a val is of type : etype

Parameters:
etype: Type 

Returns:
Boolean if it is that type
"""
def is_type(val,  etype):
    try:
        etype(val)
        return True
    except (ValueError, TypeError):
        return False
    
"""
Removes all the records that have an invalid type

Parameters:
header_df : DataFrame
col : column name to check
etype: Type 

Returns:
DataFrame with records that have valid type
"""
def rm_inval_type(df, col, etype):
    name_attr = df.name if hasattr(df, "name") else None
    df = df[df[col].apply(lambda val: is_type(val,etype))]
    if name_attr:
        df.name = name_attr
    return df

In [ ]:
"""
Removes outliers (using Interquartile range) from a given column
Parameters:
df: DataFrame
col: column to check for outliers
Returns: DataFrame with outliers removed
"""
def IQR_outlier(df, col: str):
    name_attr = df.name if hasattr(df, "name") else None
    Q1 = df[col].quantile(0.25)   
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # non_outliers = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    df_rm_outliers = df[~((df[col] < lower_bound) | (df[col] > upper_bound))]
    if name_attr:
        df_rm_outliers.name = name_attr
    return df_rm_outliers


"""
Removes outliers (using Z-score) from a given column
Parameters:
df: DataFrame
col: column to check for outliers
deviance: outlier bounds
Returns: DataFrame with outliers removed
"""
def zscore(df, col:str, deviance:int):
    col_mean = df[col].mean()
    col_std = df[col].std()
    return df[abs((df[col] - col_mean)/ col_std) <= deviance]

# LOAD
### Save into Currated layer [Silver]

In [127]:
"""
Will file "modded_{filename}.csv"
in curated-layer directory.
If path does not exist, will create it for you.

Parameters: 
df - a Data Frame

Returns: Nothing
"""
def df2csv(df):
    year = df.name[-4:]
    type = df.name[:-5]
    path = f'../data_set/CBP_AMS/curated-layer/{year}/{type}/'          
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(path + f'modded_{df.name}.csv', sep='|' , index=False) 

    """
Will file "modded_{filename}.parquet"
in curated-layer directory.
If path does not exist, will create it for you.

Parameters: 
df - a Data Frame

Returns: Nothing
"""
def df2parquet(df):
    year = df.name[-4:]
    type = df.name[:-5]
    path = f'../data_set/CBP_AMS/curated-layer/{year}/{type}/'          
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_parquet(path + f'modded_{df.name}.parquet', engine='pyarrow',index=False) 

##### For Current Use Case : We can reduce data overhead

In [ ]:
rm_billgen = ['house_bol_number','sub_house_bol_number', 'bill_type_code',
              'manifest_number', 'trade_update_date', 'run_date']

rm_container = ['seal_number_1', 'seal_number_2',
       'equipment_description_code','container_type', 'load_status']

rm_header  = ['carrier_code', 'vessel_country_code', 'vessel_name',
       'foreign_port_of_lading_qualifier', 'manifest_quantity', 'manifest_unit', 
       'measurement', 'measurement_unit', 'record_status_indicator',
       'place_of_receipt', 'port_of_destination', 'foreign_port_of_destination_qualifier', 
       'foreign_port_of_destination', 'conveyance_id_qualifier', 'conveyance_id', 
       'in_bond_entry_type', 'mode_of_transportation', 'secondary_notify_party_1',
       'secondary_notify_party_2', 'secondary_notify_party_3', 'secondary_notify_party_4', 
       'secondary_notify_party_5', 'secondary_notify_party_6', 'secondary_notify_party_7',
       'secondary_notify_party_8', 'secondary_notify_party_9', 'secondary_notify_party_10']

rm_tariff = ['description_sequence_number','harmonized_number', 'harmonized_weight_unit','harmonized_weight']


# ----------HEADER--------------

#### Cleaning Header Data

In [ ]:
"""
Convert the weight column to a universal weight unit measurement

Parameters:
header_df: DataFrame

Returns:
None
"""
def universal_unit(header_df):
    unit_weight = {
        'Kilograms': 1.0,
        'Pounds': 0.453592, 
        'Metric Ton': 1000.0, 
        'Long Ton': 1016.04691, 
        'Measurement Ton': 1.01604691
    }
    # Convert weight to kg based on weight_unit
    for unit, scale in unit_weight.items():
        header_df.loc[header_df['weight_unit'] == unit, 'weight'] *= scale
        header_df['weight_unit'] = 'Kilograms'


def is_date(val):
    try:
        dt.strptime(val, '%Y-%m-%d')
        return True
    except (ValueError, TypeError):
        return False

def rm_inval_date(df,col):
    name_attr = df.name if hasattr(df, "name") else None
    df = df[df[col].apply(is_date)]
    if name_attr:
        df.name = name_attr
    return df

"""
removes the records that have dates that are not in the scope of this year
"""
def date_outlier(df):
    name_attr = df.name if hasattr(df, "name") else None
    year = int(df.name[-4:])
    df['estimated_arrival_date'] = pd.to_datetime(df['estimated_arrival_date'], format='%Y-%m-%d')
    df['actual_arrival_date'] = pd.to_datetime(df['actual_arrival_date'], format='%Y-%m-%d')
    df = df[df['actual_arrival_date'].dt.year <= year]   # we are not in the future!!!
    df = df[(abs(df['estimated_arrival_date'].dt.year - year) <= 5)]
    df = df[(abs(df['actual_arrival_date'].dt.year - year) <= 5)]
    if name_attr:
        df.name = name_attr
    return df
    


#### Load in HEADER Data

In [ ]:
# Currated layer only requires : container |  header  |  tariff   |   billgen   
dfs = gen_dfs(path='header')
dfs[0]

In [ ]:
# Drop columns we will not be using
drop_dfs_col(dfs, rm_header)

In [ ]:
# Merge the dataframes into one 
dfs = merge_dfs(dfs)

In [ ]:
# Drop the duplicate records
rm_dupe(dfs)



#### Removal of any irregular data in the record

In [ ]:
# Check to make sure there are no null values 😀
for col in cols_with_null(dfs):
    rm_nulls(dfs, col)
# Header does not need to deal with any Nulls 🎉

In [ ]:
# check the type of weight columns 
dfs = rm_inval_type(dfs, 'weight', int)

In [ ]:
dfs = rm_inval_type(dfs, 'port_of_unlading', str)

In [ ]:
dfs = rm_inval_type(dfs, 'foreign_port_of_lading', str)

In [ ]:
dfs = rm_inval_type(dfs, 'weight_unit', str)

In [ ]:
# Removes any invalid date in this col
dfs = rm_inval_date(dfs, 'estimated_arrival_date')

In [ ]:
# Removes any invalid date in this col
dfs = rm_inval_date(dfs, 'actual_arrival_date')

In [ ]:
# Remove out the Outliers
dfs = IQR_outlier(dfs, 'weight')

In [ ]:
# remove any date outliers 
dfs = date_outlier(dfs)

In [ ]:
# Clean the weight unit to a singular unit ( Kilogram )
universal_unit(dfs)

In [ ]:
# Cleaned up Header Data -> Store into "Silver Layer" 
df2csv(dfs)

# ----------BILL GEN--------------

In [94]:
# Currated layer only requires : container |  header  |  tariff   |   billgen   
dfs = gen_dfs(path='billgen')
dfs[0]

,identifier,master_bol_number,house_bol_number,sub_house_bol_number,voyage_number,bill_type_code,manifest_number,trade_update_date,run_date
0,201901010,OOLU2593478240,NaN,NaN,091E,Master Bill,588.000,2018-12-31,2019-01-01
1,201901011,MOLU13903770302,AEHSSKOL01019,NaN,033E,House Bill,9916.000,2018-12-31,2019-01-01
2,201901012,APLU711051802,EXDO616620226,NaN,00025,House Bill,1.000,2018-03-13,2019-01-01
3,201901013,APLU711051813,EXDO616620230,NaN,00025,House Bill,1.000,2018-03-06,2019-01-01
4,201901014,APLU711051920,EXDO616620247,NaN,00025,House Bill,1.000,2018-03-19,2019-01-01
...,...,...,...,...,...,...,...,...,...
9999995,2019070888895,HLCUTYO190515489,NaN,NaN,001E,Simple BOL FROB,1.000,2019-06-25,2019-07-08
9999996,2019070888896,HLCUTYO190517770,NaN,NaN,001E,Simple BOL FROB,1.000,2019-06-25,2019-07-08
9999997,2019070888897,HLCUTYO190529030,NaN,NaN,001E,Simple BOL FROB,1.000,2019-06-25,2019-07-08
9999998,2019070888898,HLCUTYO190601508,NaN,NaN,001E,Simple BOL FROB,1.000,2019-06-25,2019-07-08


In [95]:
# Drop columns we will not be using
drop_dfs_col(dfs, rm_billgen)

In [100]:
# Merge the dataframes into one  
dfs = merge_dfs(dfs)

In [105]:
# Drop the duplicate records  
rm_dupe(dfs)

In [112]:
# remove any null values
for col in cols_with_null(dfs):
    rm_nulls(dfs, col)

identifier : False
master_bol_number : False
voyage_number : False


In [114]:
# Cleaned up Bill Gen Data -> Store into "Silver Layer" 
df2csv(dfs)

# ----------CONTAINER--------------

In [141]:
def rm_box_outlier(df):
    name_attr = df.name if hasattr(df, "name") else None
    df = df[(df['container_length'] != 0) & (df['container_width'] != 0) & (df['container_height'] != 0)]
    if name_attr:
        df.name = name_attr
    return df

In [115]:
# Currated layer only requires : container |  header  |  tariff   |   billgen   
dfs = gen_dfs(path='container')
dfs[0]

,identifier,container_number,seal_number_1,seal_number_2,equipment_description_code,container_length,container_height,container_width,container_type,load_status,type_of_service
0,201901010,OOCU6796874,OOLDZP0928,NaN,Container,4000,906,802,4EB0,Loaded,Container Yard
1,201901011,FCIU5657370,376646,NaN,Container,2000,806,800,2200,Loaded,Container Yard
2,201901012,CMAU4612647,G0137453,NaN,Container,4000,900,800,45G0,Loaded,Container Yard
3,201901012,TGHU9897308,G0137517,NaN,Container,4000,900,800,45G0,Loaded,Container Yard
4,201901012,SEGU4018985,G0137448,NaN,Container,4000,900,800,45G0,Loaded,Container Yard
...,...,...,...,...,...,...,...,...,...,...,...
9999995,2019043057226,OOLU8979700,7280342,NaN,G1,4000,900,800,45G1,Loaded,House to House
9999996,2019043057226,OOLU9568200,7280341,NaN,G1,4000,900,800,45G1,Loaded,House to House
9999997,2019043057227,OOLU7866680,OOLESU2963,NaN,G1,4000,806,800,42G1,Loaded,House to House
9999998,2019043057227,OOLU7640434,OOLESU2955,NaN,G1,4000,806,800,42G1,Loaded,House to House


In [116]:
# Drop columns we will not be using
drop_dfs_col(dfs, rm_container)

In [119]:
# Merge the dataframes into one    30635677 rows × 6 columns   27234627 rows × 6 columns
dfs = merge_dfs(dfs)

In [121]:
# Drop the duplicate records  
rm_dupe(dfs)

In [134]:
# remove any null values
for col in cols_with_null(dfs):
    rm_nulls(dfs, col)

identifier : False
container_number : False
container_length : False
container_height : False
container_width : False
type_of_service : False


In [135]:
# Removes any invalid length in this col
dfs = rm_inval_type(dfs, 'container_length', int)

In [137]:
# Removes any invalid height in this col
dfs = rm_inval_type(dfs, 'container_height', int)

In [139]:
# Removes any invalid width in this col
dfs = rm_inval_type(dfs, 'container_width', int)

In [166]:
dfs = rm_box_outlier(dfs)

In [173]:
# grouped_counts = dfs.groupby(['container_length', 'container_height', 'container_width']).size().reset_index(name='counts')


In [175]:
# grouped_counts.sort_values('counts')

,container_length,container_height,container_width,counts
54,4300,405,800,1
76,4800,906,800,1
55,4300,806,800,1
56,4500,106,800,1
58,4500,400,800,1
...,...,...,...,...
17,2000,806,802,1313479
49,4000,906,802,1966473
43,4000,806,800,2770236
16,2000,806,800,4284757


In [167]:
dfs.describe()

,identifier,container_length,container_height,container_width
count,25707759.000,25707759.000,25707759.000,25707759.000
mean,1719988920255.021,3575.997,867.265,800.423
std,685883612136.753,843.788,47.140,3.572
min,201901010.000,6.000,9.000,8.000
25%,2019020618414.000,4000.000,806.000,800.000
50%,2019052750135.000,4000.000,900.000,800.000
75%,2019090942334.500,4000.000,900.000,800.000
max,20191130100288.000,6200.000,2400.000,1306.000


In [ ]:
df2parquet(dfs)

#### Function to Export Modified Data 

In [ ]:
"""
Establish a connection to the MySQL database

Returns: 
"""
def link_db():
    try:
        return pymssql.connect(server='DESKTOP-N9SA336')
    except:
        return None


In [ ]:
conn = link_db()
if conn:
    cursor = conn.cursor() # Cursors are database level objects that let your query a database multiple times - Think of it as a pointer to a row
    cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES")
    rows = cursor.fetchall()
    [print(row) for row in rows]